## 1. 라이브러리 설치 및 불러오기  
YOLO 모델을 실행하기 위한 환경을 구성하는 코드입니다.  
필요한 패키지를 설치하고, 이미지 처리 및 데이터 로딩에 필요한 라이브러리를 불러옵니다.

> ✅ 설치 패키지
- `ultralytics`: YOLO 모델 실행 및 학습
- `opencv-python`: 이미지 로딩 및 전처리
- `numpy`, `matplotlib`: 데이터 처리 및 시각화

> ✅ 불러오는 핵심 라이브러리
- `torch`, `torchvision`: 모델 학습 및 데이터 전처리
- `ultralytics.YOLO`: YOLO 모델 로딩
- `cv2`, `numpy`: 이미지 처리 및 수치 계산
- `tqdm`, `matplotlib.pyplot`: 시각화 및 진행 표시

In [ ]:
%pip install ultralytics opencv-python numpy matplotlib tqdm pyyaml -q

import os
import cv2
import torch
import yaml
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from ultralytics import YOLO
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

## 2. 랜덤 시드 고정

실험의 일관성을 유지하기 위해 `random`, `numpy`, `torch`의 시드를 고정합니다.  

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
seed_everything()

## 3. YOLO 모델 학습  
모델은 yolov8m.pt를 사용, 전처리는 학습 과정에서 직접 조작하며 학습하도록 함.

### Kaggle에서 제공하는 30시간으로 부족하여 데이터를 팀장 개인 데스크탑에 다운로드 받은 후 로컬 환경에서 진행하였습니다.

In [ ]:
from ultralytics import YOLO

# 1. 모델 로딩 (YOLOv9-M)
model = YOLO("yolov8m.pt")

# 2. 훈련 함수 정의
def train_model():
    results = model.train(
        data=r"G:\AI project\SSAFY_AI_CHALLENGE\data.yaml",  # 데이터셋 yaml 경로
        epochs=30,
        imgsz=768,                    # YOLOv9에 적합한 크기로 업
        batch=8,                      # VRAM 고려해서 8로 설정
        workers=4,
        device="0",
        amp=True,

        # 프로젝트 경로 및 이름 설정
        project="runs/train",
        name="pothole_yolov8m_v1",
        exist_ok=False,

        # 학습률 및 옵티마이저 설정
        lr0=0.0015,                  
        lrf=0.0003,
        optimizer="SGD",
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3,
        cos_lr=True,

        # 손실 가중치
        box=6.0,
        cls=0.1,
        dfl=1.5,

        # 데이터 증강 (너무 강하지 않게)
        mosaic=0.7,
        mixup=0.05,
        copy_paste=0.05,
        hsv_h=0.015,
        hsv_s=0.5,
        hsv_v=0.3,
        translate=0.05,
        scale=0.3,
        fliplr=0.5,
        perspective=0.0002,

        # 기타 옵션
        overlap_mask=True,
        rect=False,
        close_mosaic=15,
        patience=20,
        val=True,
        plots=True
    )
    return results

# 3. 학습 실행
results = train_model()


## 10. 제출 파일 생성 (submission.csv)
테스트 이미지에 대해 학습된 YOLO 모델을 사용하여 객체 탐지를 수행하고,  
예측 결과를 제출 형식에 맞춰 `submission.csv` 파일로 저장합니다.

conf 0.05, imgsz 640으로 수정하였습니다.

In [ ]:
import pandas as pd

model = YOLO(r"G:\AI project\SSAFY_AI_CHALLENGE\runs\train\pothole_yolov8m_v1\weights\best.pt")

TEST_IMG_DIR = r"G:\AI project\SSAFY_AI_CHALLENGE\test\images"
test_image_paths = sorted(glob(os.path.join(TEST_IMG_DIR, "*.jpg")))

submission_rows = []

for img_path in test_image_paths:
    image_id = os.path.basename(img_path)

    if cv2.imread(img_path) is None:
        print(f"이미지 로드 실패: {image_id}")
        submission_rows.append({
            "ImageId": image_id,
            "ClassId": 0,
            "X": 0,
            "Y": 0,
            "Width": 0,
            "Height": 0,
        })
        continue

    results = model.predict(source=img_path, conf=0.05, imgsz=528, save=False)
    result = results[0]

    if len(result.boxes) > 0:
        boxes = result.boxes
        best_idx = boxes.conf.argmax().item()
        cls_id = int(boxes.cls[best_idx].item())
        cx, cy, w, h = boxes.xywhn[best_idx].tolist()

        submission_rows.append({
            "ImageId": image_id,
            "ClassId": cls_id,
            "X": round(cx, 6),
            "Y": round(cy, 6),
            "Width": round(w, 6),
            "Height": round(h, 6),
        })
    else:
        submission_rows.append({
            "ImageId": image_id,
            "ClassId": 0,
            "X": 0,
            "Y": 0,
            "Width": 0,
            "Height": 0,
        })

submission_df = pd.DataFrame(submission_rows, columns=["ImageId", "ClassId", "X", "Y", "Width", "Height"])
submission_path = "G:\AI project\결과 사진\submission4.csv"
submission_df.to_csv(submission_path, index=False)
print(f"제출 파일 저장 완료: {submission_path}")